In [1]:
%load_ext autoreload  
%autoreload 2 
%matplotlib inline

In [2]:
import torch 
from tqdm import tqdm
import json
from datasets import load_dataset
from sentence_transformers import CrossEncoder

In [3]:
# get instances
dataset = load_dataset("tab_fact", 'tab_fact')

Downloading: 197MB [11:26, 286kB/s]


Dataset tab_fact downloaded and prepared to /Users/garylai/.cache/huggingface/datasets/tab_fact/tab_fact/1.0.0/bcae1c44400dec0f74f22bba297d6fe979e9751fb57d4e421f9d1676086c2985. Subsequent calls will reuse this data.


100%|██████████| 3/3 [00:00<00:00, 222.25it/s]


In [9]:
dataset = dataset['train']

In [19]:
len(dataset)

92283

In [37]:
dataset[:2]

{'id': [0, 0],
 'table_id': ['2-15401676-3.html.csv', '2-15401676-3.html.csv'],
 'table_text': ['date#result#score#brazil scorers#competition\nmay 11 , 1919#w#6 - 0#friedenreich (3) , neco (2) , haroldo#south american championship\nmay 18 , 1919#w#3 - 1#heitor , amílcar , millon#south american championship\nmay 26 , 1919#d#2 - 2#neco (2)#south american championship\nmay 29 , 1919#w#1 - 0#friedenreich#south american championship\njune 1 , 1919#d#3 - 3#haroldo , arlindo (2)#taça roberto cherry\n',
  'date#result#score#brazil scorers#competition\nmay 11 , 1919#w#6 - 0#friedenreich (3) , neco (2) , haroldo#south american championship\nmay 18 , 1919#w#3 - 1#heitor , amílcar , millon#south american championship\nmay 26 , 1919#d#2 - 2#neco (2)#south american championship\nmay 29 , 1919#w#1 - 0#friedenreich#south american championship\njune 1 , 1919#d#3 - 3#haroldo , arlindo (2)#taça roberto cherry\n'],
 'table_caption': ['1919 in brazilian football', '1919 in brazilian football'],
 'statement

In [31]:
dataset[247]

{'id': 38,
 'table_id': '2-1641054-2.html.csv',
 'table_text': "university#location#established#endowment as of 2008#campus area (acres)#kiplinger 's top 100 values#enrollment as of 2008\nflorida a&m university#tallahassee , florida#1887#119 million#419#n / a#11567\nflorida atlantic university#boca raton , florida#1961#182 million#850#n / a#26525\nflorida gulf coast university#fort myers , florida#1991#39 million#760#n / a#9387\nflorida international university#miami , florida#1965#97 million#573#n / a#38614\nflorida state university#tallahassee , florida#1851#570 million#1200#17th overall in the united states#41002\nnew college of florida#sarasota , florida#1960#33 million#144#8th overall in the united states#769\nuniversity of central florida#orlando , florida#1963#114 million#1415#42nd overall in the united states#48699\nuniversity of florida#gainesville , florida#1853#1.3 billion#2000#2nd overall in the united states#52084\nuniversity of north florida#jacksonville , florida#1969#95

In [38]:
def get_instance(index, dataset):  
    return {
        "input": dataset['table_text'][index],
        "output": dataset['statement'][index]
    }

In [50]:
Instances = []
for i in tqdm(range(21, 1000)):
    if i != 247 and dataset['label'][i] == 1:
        instance = get_instance(i, dataset)
        Instances.append(instance)

100%|██████████| 979/979 [04:24<00:00,  3.71it/s]


In [46]:
# Instances

In [15]:
task_json = {
    "Contributors": [
        "Gary Haizhi Lai"
    ],
    "Source": [
        "tab_fact (https://openreview.net/forum?id=rkeJRhNYDH)"
    ],
    "Categories": [
        "Text Generation"
    ],
    "Definition": "In this task, we ask you look at a table and generate a fact entailed by information in the table. All words should be lowercase. Write all the nouns in singular form (e.g. \"games\" becomes \"game\"). Write all the verbs in infinitive form (e.g. \"is\" and \"are\" become \"be\"; \"used\" becomes \"use\")",
    "Positive Examples": [
        {
            "input": dataset[0]['table_text'],
            "output": "4 of the 5 game be play in may 1919",
            "explanation": "The output fact is supported by the input table. The verbs, nouns and capitalization are in the correct format."
        },
        {
            "input": dataset[247]['table_text'],
            "output": dataset[247]['statement'],
            "explanation": "The output fact is correct according to the input table. The verbs, nouns and capitalization are in the correct format."
        }
    ],
    "Negative Examples": [
        {
            "input": dataset[0]['table_text'],
            "output": "5 of the 5 game be play in may 1919",
            "explanation": "The fact conflicts with the input table. It should be 4 of the 5 game."
        },
        {
            "input": dataset[247]['table_text'],
            "output": "New College of Florida has the smallest endowment of any university with only 33 millions",
            "explanation": "While the fact is supported by the input table, the format is wrong. There are plural nouns, verbs not in infinitive form, and capitalization."
        }
    ],
    "Instances": Instances
}

# export
with open('task1200_tab_fact_text_generation.json', 'w') as fp:
    final_json = json.dumps(task_json, indent=4, ensure_ascii=False)
    print(final_json, file=fp)